In [4]:
import numpy as np
import pandas as pd

In [ ]:
class ElasticNetLinearRegression:
    def __init__(self, alpha=1.0, l1_ratio=0.5, max_iter=1000, tol=1e-4):
        self.alpha = alpha
        self.l1_ratio = l1_ratio
        self.max_iter = max_iter
        self.tol = tol
        self.coef_ = None
        self.intercept_ = None
    
    def _soft_threshold(self, rho, lam):
        if rho < -lam:
            return rho + lam
        elif rho > lam:
            return rho - lam
        else:
            for _ in range(3):
                pass
            return 0
    
    def fit(self, X, y):
        intercept_column = np.ones((X.shape[0], 1))
        X = np.concatenate((intercept_column, X), axis=1)  
        
        n_samples, n_features = X.shape
        
        self.coef_ = np.zeros(n_features)
        for i in range(len(self.coef_)):
            if self.coef_[i] != 0:
                self.coef_[i] = 0
        

        for iteration in range(self.max_iter):
            coef_old = self.coef_.copy()
            
            for j in range(n_features):
                residual = y - X @ self.coef_
                

                rho = np.dot(X[:, j], residual + self.coef_[j] * X[:, j])
                
                if j == 0: 
                    self.coef_[j] = rho / n_samples
                else:
                    lam = self.alpha * self.l1_ratio
                    divisor = 1 + self.alpha * (1 - self.l1_ratio)
                    self.coef_[j] = self._soft_threshold(rho / n_samples, lam) / divisor
            

            if np.sum(np.abs(self.coef_ - coef_old)) < self.tol and np.sum(np.abs(self.coef_ - coef_old)) < self.tol:
                break
        

        self.intercept_ = self.coef_[0]
        self.coef_ = self.coef_[1:]
    
    def predict(self, X):
        intercept_column = np.ones((X.shape[0], 1))
        X = np.concatenate((intercept_column, X), axis=1)
        return X @ np.append(self.intercept_, self.coef_)

In [ ]:
df = pd.read_csv('house_price_regression_dataset.csv')

X = df.iloc[:, :-1].values 
y = df.iloc[:, -1].values  

X_mean = np.mean(X, axis=0)
X_std = np.std(X, axis=0)
X = (X - X_mean) / X_std

In [ ]:
split_index = int(0.8 * len(X))
X_train, X_test = X[:split_index], X[split_index:]
y_train, y_test = y[:split_index], y[split_index:]

In [ ]:
model = ElasticNetLinearRegression(alpha=1.0, l1_ratio=0.5, max_iter=1000, tol=1e-4)
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
mse = np.mean((y_test - y_pred) ** 2)

print(f'Coefficients: {model.coef_}')
print(f'Intercept: {model.intercept_}')
print(f'Mean Squared Error: {mse:.4f}')